In [1]:
import tweepy
from decouple import config

In [2]:
TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)

In [3]:
import re
from collections import Counter

In [4]:
def check_num_tweets(search):
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=2000,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    return tweets

In [5]:
t = check_num_tweets("webdevmason")

In [6]:
len(t)

200

In [7]:
def get_first_user_interactions(search):
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=200,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    interactions = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    #counter returns an object with a count for each account
    interactions = [ i[0] for i in Counter(interactions).most_common(10) ]
    interactions.append(search)
    return interactions


In [8]:
a = get_first_user_interactions("webdevmason")

In [9]:
a

['kevinsimler',
 'visakanv',
 'austen',
 'joefyhrceesay',
 'paulg',
 'spakhm',
 'homsit',
 'jonathan_blow',
 'andy_matuschak',
 'tristanharris',
 'webdevmason']

In [10]:
first = Counter(a).most_common(10)

In [11]:
first

[('kevinsimler', 1),
 ('visakanv', 1),
 ('austen', 1),
 ('joefyhrceesay', 1),
 ('paulg', 1),
 ('spakhm', 1),
 ('homsit', 1),
 ('jonathan_blow', 1),
 ('andy_matuschak', 1),
 ('tristanharris', 1)]

In [20]:
def get_first_user_connections(search):
    twitter_user = TWITTER.get_user(search)
    tweets = twitter_user.timeline(
                count=1000,
                exclude_replies=False,
                include_rts=True,
                tweet_mode='extended'
    )
    b = [ i.full_text for i in tweets ]
    b = " ".join(b)
    b = b.lower()
    b = b.replace(search, "")
    index = re.findall(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)',b)
    interactions = " ".join(index)
    interactions_list.append(interactions)
    index.append(search)
    return interactions

In [21]:
type(first)

list

In [22]:
interactions_list = []
index = []
get_first_user_connections("austen")
for i in range(len(a)):
    try:
        get_first_user_connections(a[i][0])
    except:
        pass

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = interactions_list
vectorizer = CountVectorizer(min_df=2)
X = vectorizer.fit_transform(corpus)

In [24]:
import pandas as pd
pd.set_option("display.max_columns", 100)
display_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index=index)

ValueError: Shape of passed values is (288, 10), indices imply (288, 0)

In [ ]:
display_df

In [ ]:
import numpy as np

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
DG=nx.MultiGraph()

#loop through index, then the columns to find non-zero connections.
for i in display_df.index.values:
    for j in display_df.columns:
        if display_df[j].loc[i] >0 :
            DG.add_edge(j,i, weight = np.sqrt(display_df[j].loc[i] ))
        else:
            pass
        



In [ ]:
test_a = get_first_user_connections("webdevmason")

In [ ]:
sorted(str.split(test_a, " "))

In [ ]:
edges = nx.to_pandas_edgelist(DG)

In [ ]:
nx.draw_kamada_kawai(DG,node_size=20)

In [ ]:
fig = plt.figure(figsize=(8,8))
nx.draw_spectral(DG,node_size=20)
plt.show()

## trying k-nearest neighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
sl_nodes = nx.spectral_layout(DG)

In [ ]:
sl_nodes["alaskaair"]

In [ ]:
sl_nodes

In [ ]:
def get_column_sum(i):
    try:
        return display_df[i].sum()
    except:
        return 1

In [ ]:
node_weights = [ get_column_sum(i) for i in list(sl_nodes)]

In [ ]:
node_weights

In [ ]:
neigh = NearestNeighbors(2, 0.4)
neigh.fit(sl_nodes) 

## experimenting with cliques

In [ ]:
from networkx.algorithms.community import k_clique_communities
c = list(k_clique_communities(DG, 4))

## experimenting with edges and weights

In [ ]:
edges = list(DG.edges.data())
"""edges": [
    {
      "id": "e0",
      "source": "n0",
      "target": "n1"
    },
    {
      "id": "e1",
      "source": "n1",
      "target": "n2"
    },
    {
      "id": "e2",
      "source": "n2",
      "target": "n0"
    }
  ]
}"""

In [ ]:
#edges_dict = [ {"source": edge[0],"target":edge[1], "weight" : edge[2]["weight"]} for edge in edges]
edges_dict = [ {"id": edge[0]+" "+ edge[1],"source": edge[0],"target":edge[1]} for edge in edges]

In [ ]:
edge_json = { "edges" : edges_dict}

In [ ]:
edge_json

In [ ]:
rows = [edges_dict[i] for i in range(0,len(edges_dict)) if
        (edges_dict[i]["target"]== "visakanv")|
        (edges_dict[i]["target"]== "paulg")|
        (edges_dict[i]["target"]=="austen") ]


In [ ]:
rows2 = sorted(rows, key=lambda k: k["weight"],reverse=True)

In [ ]:
rows2

In [ ]:
"""{
  "nodes": [
    {
      "id": "n0",
      "label": "A node",
      "x": 0,
      "y": 0,
      "size": 3
    },
    {
      "id": "n1",
      "label": "Another node",
      "x": 3,
      "y": 1,
      "size": 2
    },"""

In [ ]:
nodes_dict = [ {"id": i, "label": i, "x": sl_nodes[i][0], "y":sl_nodes[i][1] } for i in list(sl_nodes)]

In [ ]:
nodes_dict

In [ ]:
data = (nodes_json, edge_json)

In [ ]:
import json

In [ ]:
with open('data.json', 'w') as outfile:
    json.dump(nodes_dict, outfile)

In [ ]:
edges[0][2]["weight"]

In [ ]:
sl_nodes[edges[0][0]][0]
sl_nodes[edges[0][0]][1]

In [ ]:
node_list = list(sl_nodes)
node_x = [sl_nodes[i][0] for i in node_list]
node_y = [sl_nodes[i][1] for i in node_list]

In [ ]:
sl_edges_x = []
sl_edges_y = []
sl_edges_weight = []
for i in range(len(edges)):
    sl_edges_x.append(sl_nodes[edges[i][0]][0])
    sl_edges_y.append(sl_nodes[edges[i][0]][1])
    sl_edges_x.append(sl_nodes[edges[i][1]][0])
    sl_edges_y.append(sl_nodes[edges[i][1]][1])
    sl_edges_weight.append(edges[i][2]["weight"])
    sl_edges_weight.append(edges[i][2]["weight"])
    sl_edges_x.append("")
    sl_edges_y.append("")
    sl_edges_weight.append("")

In [ ]:
sl_edges_x

## experimenting with plotting json, js, etc

In [ ]:
import json

In [ ]:
x = json.dumps(sl_edges_x)
y = json.dumps(sl_edges_y)
weights = json.dumps(sl_edges_weight)

In [ ]:
print(x)

In [ ]:
with open('x.json', 'w') as outfile:
    json.dump(x, outfile)
with open('y.json', 'w') as outfile:
    json.dump(y, outfile)
with open('weights.json', 'w') as outfile:
    json.dump(weights, outfile)

In [ ]:
#!pip install pixiedust --upgrade
#!pip install pixiedust_node --upgrade

In [ ]:
#import pixiedust_node

In [ ]:
%%node
var d_x = fs.readFileSync("E:/ds/labs12-twitter-blocker-DS/x.json");
var d_y = fs.readFileSync("E:/ds/labs12-twitter-blocker-DS/y.json");
var d_w = fs.readFileSync("E:/ds/labs12-twitter-blocker-DS/weights.json");
console.log(d_x)

## Matrix Multiplication
Really don't need the graph library 

In [ ]:
x2 = X @ X.T
pd.DataFrame(x2.toarray(), columns = index, index = index)